In this piece of code I simultaneously optimize over the unitary basis rotation and the mps tensors.

In [1]:
using Revise,LinearAlgebra, MPSKit, TensorKit, MPSKitExperimental

LinearAlgebra.BLAS.set_num_threads(1);
TensorKit.TO.disable_cache();
TensorKit.Strided.disable_threads();

(ERI,K,E0,NORB,NELEC,MS2) = parse_fcidump("N2.CCPVDZ.FCIDUMP")

([1.1486574428316065 + 0.0im 0.09862429942213358 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.09862429942213358 + 0.0im 0.01772802215378584 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 0.009036810489967113 + 0.0im 0.003653054088686124 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.003653054088686124 + 0.0im 0.0017533811663050644 + 0.0im;;; 0.09862429942213358 + 0.0im 0.413306086967678 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.01772802215378584 + 0.0im -0.002611637416218654 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … -0.00533684879644213 + 0.0im 0.002227779206976413 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … -0.001273700295269154 + 0.0im -0.0005596210416448279 + 0.0im;;; 0.03622757258934596 + 0.0im -0.0470908045818145 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0013749347300627281 + 0.0im 0.004411244674669814 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … -2.8000517192292658e-5 + 0.0im -0.006213347982843338 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … -0.000871798277289064 + 

I will keep the first 3 orbitals frozen filled, treat the next 10 orbitals using MPS, and keep the rest frozen free.

In [2]:
N_frozen_filled = 3;
N_active = 10;
N_frozen_empty = NORB-N_frozen_filled-N_active;

The state is now smaller:

In [3]:
psp = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((0,0,0)=>1, (1,1//2,1)=>1, (2,0,0)=>1);

left = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((-(NELEC-N_frozen_filled*2),MS2//2,mod1(-(NELEC-N_frozen_filled*2),2))=>1);
right = oneunit(left);

virtual = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((i,s,b)=>1 for i in -(NELEC-N_frozen_filled*2):0, s in 0:1//2:(MS2//2+1), b in (0,1));

st = FiniteMPS(rand,ComplexF64,N_active,psp,virtual;left,right);

The code works through a weird new type:

In [4]:
nh = CASSCF_Ham(ERI,K,E0,Matrix(qr(randn(size(K,1),size(K,1))).Q),N_frozen_filled+1:N_frozen_filled+N_active)

CASSCF_Ham{Array{ComplexF64, 4}, Matrix{ComplexF64}, ComplexF64, Matrix{Float64}, UnitRange{Int64}}([1.1486574428316065 + 0.0im 0.09862429942213358 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.09862429942213358 + 0.0im 0.01772802215378584 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 0.009036810489967113 + 0.0im 0.003653054088686124 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.003653054088686124 + 0.0im 0.0017533811663050644 + 0.0im;;; 0.09862429942213358 + 0.0im 0.413306086967678 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.01772802215378584 + 0.0im -0.002611637416218654 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … -0.00533684879644213 + 0.0im 0.002227779206976413 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … -0.001273700295269154 + 0.0im -0.0005596210416448279 + 0.0im;;; 0.03622757258934596 + 0.0im -0.0470908045818145 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0013749347300627281 + 0.0im 0.004411244674669814 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … -2.8000517192

Let's try it out!

In [7]:
mpo_opp = MPSKitExperimental.mpo_representation(nh)[1]; # will be made easier in future
(st,env) = find_groundstate(st,mpo_opp,DMRG2(maxiter=2,trscheme=truncdim(50)),disk_environments(st,mpo_opp));
(st,nh) = find_groundstate(st,nh,GrassmannSCF(maxiter=10));

┌ Info: Iteraton 0 error 0.5259220245893251
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:112


┌ Info: Iteraton 1 error 0.16241231220703967
└ @ MPSKit /home/maavdamm/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:112


┌ Warning: not a descent direction, increasing shift -11.437526758819 + 2.858824288409778e-15im 0+33.33406335758989
└ @ MPSKitExperimental /home/maavdamm/MPSKitExperimental.jl/src/quantumchemistry/grassmann_scf.jl:1324
┌ Info: CG: initializing with f = -62.525663980251, ‖∇f‖ = 3.3355e+01
└ @ OptimKit /home/maavdamm/.julia/packages/OptimKit/xpmbV/src/cg.jl:39


┌ Warning: not a descent direction, increasing shift -11.437526758819 + 2.858824288409778e-15im 0+33.33406335758989
└ @ MPSKitExperimental /home/maavdamm/MPSKitExperimental.jl/src/quantumchemistry/grassmann_scf.jl:1324


┌ Info: CG: iter    1: f = -71.574167933731, ‖∇f‖ = 2.4368e+01, α = 2.00e+00, β = 0.00e+00, nfg = 3
└ @ OptimKit /home/maavdamm/.julia/packages/OptimKit/xpmbV/src/cg.jl:83


...

┌ Info: CG: iter    7: f = -92.050712011388, ‖∇f‖ = 1.4353e+01, α = 1.86e-01, β = 1.35e+00, nfg = 5
└ @ OptimKit /home/maavdamm/.julia/packages/OptimKit/xpmbV/src/cg.jl:83


┌ Info: CG: iter    8: f = -95.444911502514, ‖∇f‖ = 1.3340e+01, α = 2.85e-01, β = 2.48e-01, nfg = 2
└ @ OptimKit /home/maavdamm/.julia/packages/OptimKit/xpmbV/src/cg.jl:83


┌ Info: CG: iter    9: f = -97.958645899643, ‖∇f‖ = 1.2705e+01, α = 2.42e-01, β = 4.65e-01, nfg = 3
└ @ OptimKit /home/maavdamm/.julia/packages/OptimKit/xpmbV/src/cg.jl:83


┌ Warning: CG: not converged to requested tol: f = -102.079435800751, ‖∇f‖ = 9.5152e+00
└ @ OptimKit /home/maavdamm/.julia/packages/OptimKit/xpmbV/src/cg.jl:103


(FiniteMPS{TensorMap{GradedSpace{TensorKit.ProductSector{Tuple{U1Irrep, SU2Irrep, Fermion{TensorKit._fermionparity, Z2Irrep}}}, TensorKit.SortedVectorDict{TensorKit.ProductSector{Tuple{U1Irrep, SU2Irrep, Fermion{TensorKit._fermionparity, Z2Irrep}}}, Int64}}, 2, 1, TensorKit.ProductSector{Tuple{U1Irrep, SU2Irrep, Fermion{TensorKit._fermionparity, Z2Irrep}}}, TensorKit.SortedVectorDict{TensorKit.ProductSector{Tuple{U1Irrep, SU2Irrep, Fermion{TensorKit._fermionparity, Z2Irrep}}}, Matrix{ComplexF64}}, FusionTree{TensorKit.ProductSector{Tuple{U1Irrep, SU2Irrep, Fermion{TensorKit._fermionparity, Z2Irrep}}}, 2, 0, 1, Nothing}, FusionTree{TensorKit.ProductSector{Tuple{U1Irrep, SU2Irrep, Fermion{TensorKit._fermionparity, Z2Irrep}}}, 1, 0, 0, Nothing}}, TensorMap{GradedSpace{TensorKit.ProductSector{Tuple{U1Irrep, SU2Irrep, Fermion{TensorKit._fermionparity, Z2Irrep}}}, TensorKit.SortedVectorDict{TensorKit.ProductSector{Tuple{U1Irrep, SU2Irrep, Fermion{TensorKit._fermionparity, Z2Irrep}}}, Int64}}